# Development notebook
> Figuring out how API calls work and such

In [18]:
# Basic imports; set API key

import openai
import re

openai.api_key_path = "api_key"


In [3]:
# Global variables

MODEL = "gpt-4"
SOURCE_LANGUAGE = "English"
TARGET_LANGUAGE = "Russian"


In [32]:
# Turn file into JSON:

# Read input
full_text = open("jabberwocky.txt", "r").read()

# Parse file into sentences:
full_text = full_text.replace("\n", "\n<NEWLINE>\n")
full_text = full_text.replace("\t", "TAB")
sentences = re.split(r"[.!\n]", full_text)
sentences = [s.strip() for s in sentences]
sentences = [s for s in sentences if len(s) > 0]

# Turn sentences into JSON
# sentences = str(sentences)
sentences


['’Twas brillig, and the slithy toves',
 '<NEWLINE>',
 'Did gyre and gimble in the wabe:',
 '<NEWLINE>',
 'All mimsy were the borogoves,',
 '<NEWLINE>',
 'And the mome raths outgrabe',
 '<NEWLINE>',
 '<NEWLINE>',
 '“Beware the Jabberwock, my son',
 '<NEWLINE>',
 'The jaws that bite, the claws that catch',
 '<NEWLINE>',
 'Beware the Jubjub bird, and shun',
 '<NEWLINE>',
 'The frumious Bandersnatch',
 '”',
 '<NEWLINE>',
 '<NEWLINE>',
 'He took his vorpal sword in hand;',
 '<NEWLINE>',
 'Long time the manxome foe he sought—',
 '<NEWLINE>',
 'So rested he by the Tumtum tree',
 '<NEWLINE>',
 'And stood awhile in thought',
 '<NEWLINE>',
 '<NEWLINE>',
 'And, as in uffish thought he stood,',
 '<NEWLINE>',
 'The Jabberwock, with eyes of flame,',
 '<NEWLINE>',
 'Came whiffling through the tulgey wood,',
 '<NEWLINE>',
 'And burbled as it came',
 '<NEWLINE>',
 '<NEWLINE>',
 'One, two',
 'One, two',
 'And through and through',
 '<NEWLINE>',
 'The vorpal blade went snicker-snack',
 '<NEWLINE>',
 'He

In [12]:
# Prompt

prompt = f"""
Please translate the following text from {SOURCE_LANGUAGE} to {TARGET_LANGUAGE}.
Sentences are provided as a JSON list of sentences. Please translate each
sentence individually and return a JSON list of translated sentences. The token
<NEWLINE> represents a newline character and should be transcribed verbatim.
"""


In [19]:
# API call
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": sentences},
    ],
    temperature=0.0,
)
response


<OpenAIObject chat.completion id=chatcmpl-7pOMwKMS8Y1eDsJ6OiKoWS9j7u5A8 at 0x7f853b223890> JSON: {
  "id": "chatcmpl-7pOMwKMS8Y1eDsJ6OiKoWS9j7u5A8",
  "object": "chat.completion",
  "created": 1692482758,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "['\u0411\u044b\u043b \u0432\u0435\u0447\u0435\u0440\u043e\u043a, \u0438 \u0441\u043a\u043e\u043b\u044c\u0437\u043a\u0438\u0435 \u0442\u043e\u0432\u044b', '<NEWLINE>', '\u0412 \u0432\u0430\u0431\u0435 \u043a\u0440\u0443\u0436\u0438\u043b\u0438\u0441\u044c \u0438 \u0432\u0435\u0440\u0442\u0435\u043b\u0438\u0441\u044c:', '<NEWLINE>', '\u0412\u0441\u0435 \u043c\u0438\u043c\u0437\u0438 \u0431\u044b\u043b\u0438 \u0431\u043e\u0440\u043e\u0433\u043e\u0432\u044b,', '<NEWLINE>', '\u0418 \u043c\u043e\u043c\u044b \u0440\u0430\u0444\u044b \u0432\u044b\u0433\u0440\u0435\u0431\u0430\u043b\u0438', '<NEWLINE>', '<NEWLINE>', '\u201c\u041e\u0441\u0442\u0435\u0440\u0435\u043

In [26]:
print(response["choices"][0]["message"]["content"])

['Был вечерок, и скользкие товы', '<NEWLINE>', 'В вабе кружились и вертелись:', '<NEWLINE>', 'Все мимзи были бороговы,', '<NEWLINE>', 'И момы рафы выгребали', '<NEWLINE>', '<NEWLINE>', '“Остерегайся Джаббервока, мой сын', '<NEWLINE>', 'Челюсти, что кусают, когти, что ловят', '<NEWLINE>', 'Остерегайся птицы Джубджуб, и избегай', '<NEWLINE>', 'Гневного Бандерснатча', '”', '<NEWLINE>', '<NEWLINE>', 'Он взял свой ворпал-меч в руки;', '<NEWLINE>', 'Долго искал он врага манксома—', '<NEWLINE>', 'Так отдохнул он под деревом Тамтам', '<NEWLINE>', 'И стоял некоторое время в раздумье', '<NEWLINE>', '<NEWLINE>', 'И, стоя в уффишной задумчивости,', '<NEWLINE>', 'Джаббервок, с глазами пламенными,', '<NEWLINE>', 'Пришуршал сквозь тулги лес,', '<NEWLINE>', 'И бурчал, как пришел', '<NEWLINE>', '<NEWLINE>', 'Раз, два', 'Раз, два', 'И взад и вперед', '<NEWLINE>', 'Ворпал-лезвие шмыг-шмыг', '<NEWLINE>', 'Он оставил его мертвым, и с его головой', '<NEWLINE>', 'Он вернулся галамфируя', '<NEWLINE>', '<NEWLI

In [33]:
# Turn sentences array into chunks so we don't exceed the max token limit
from transformers import GPT2TokenizerFast

chunks = []
chunk = []
chunk_length = 0
total_tokens = 0
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
for sentence in sentences:
    n_tokens = len(tokenizer(sentence)["input_ids"])
    if chunk_length + n_tokens > 4096:
        chunks.append(chunk)
        chunk = []
        total_tokens += chunk_length
        chunk_length = 0
    chunk.append(sentence)
    chunk_length += n_tokens
chunks.append(chunk)  # Add the last chunk
total_tokens += chunk_length
print(
    f"Broke input into {len(chunks)} chunks spanning {len(sentences)} sentences and {total_tokens} tokens.",
)
chunks

Broke input into 1 chunks spanning 67 sentences and 411 tokens.


[['’Twas brillig, and the slithy toves',
  '<NEWLINE>',
  'Did gyre and gimble in the wabe:',
  '<NEWLINE>',
  'All mimsy were the borogoves,',
  '<NEWLINE>',
  'And the mome raths outgrabe',
  '<NEWLINE>',
  '<NEWLINE>',
  '“Beware the Jabberwock, my son',
  '<NEWLINE>',
  'The jaws that bite, the claws that catch',
  '<NEWLINE>',
  'Beware the Jubjub bird, and shun',
  '<NEWLINE>',
  'The frumious Bandersnatch',
  '”',
  '<NEWLINE>',
  '<NEWLINE>',
  'He took his vorpal sword in hand;',
  '<NEWLINE>',
  'Long time the manxome foe he sought—',
  '<NEWLINE>',
  'So rested he by the Tumtum tree',
  '<NEWLINE>',
  'And stood awhile in thought',
  '<NEWLINE>',
  '<NEWLINE>',
  'And, as in uffish thought he stood,',
  '<NEWLINE>',
  'The Jabberwock, with eyes of flame,',
  '<NEWLINE>',
  'Came whiffling through the tulgey wood,',
  '<NEWLINE>',
  'And burbled as it came',
  '<NEWLINE>',
  '<NEWLINE>',
  'One, two',
  'One, two',
  'And through and through',
  '<NEWLINE>',
  'The vorpal bla

In [34]:
# API call:
responses = []
for chunk in chunks:
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": str(chunk)},
            # str(chunk) is JSON-formatted by default
        ],
        temperature=0.0,
    )
    response_message = response["choices"][0]["message"]["content"]
    responses.append(response_message)

    # Check for weirdness
    if response["choices"][0]["finish_reason"] == "max_tokens":
        print(
            f"WARNING: Reached max tokens for chunk {chunk}. Translation may be incomplete.",
            file=sys.stderr,
        )

responses

["['Был вечерок, и скользкие товы', '<NEWLINE>', 'Вращались и кружились в вабе:', '<NEWLINE>', 'Все мимзи были бороговы,', '<NEWLINE>', 'И момы раты выгребали', '<NEWLINE>', '<NEWLINE>', '“Остерегайся Жаббервока, мой сын', '<NEWLINE>', 'Челюсти, что кусают, когти, что ловят', '<NEWLINE>', 'Остерегайся птицы Юб-юб, и избегай', '<NEWLINE>', 'Зловещего Бандершнатча', '”', '<NEWLINE>', '<NEWLINE>', 'Он взял свой ворпаловый меч в руки;', '<NEWLINE>', 'Долго искал он врага манксома—', '<NEWLINE>', 'Так отдыхал он у дерева Тамтам', '<NEWLINE>', 'И стоял некоторое время в раздумье', '<NEWLINE>', '<NEWLINE>', 'И, стоя в уффишной задумчивости,', '<NEWLINE>', 'Жаббервок, с глазами пламени,', '<NEWLINE>', 'Пришуршал сквозь тулги лес,', '<NEWLINE>', 'И бурчал, как приходил', '<NEWLINE>', '<NEWLINE>', 'Раз, два', 'Раз, два', 'И взад и вперед', '<NEWLINE>', 'Ворпаловый клинок шмыг-шмяк', '<NEWLINE>', 'Он оставил его мертвым, и с его головой', '<NEWLINE>', 'Он вернулся галамфируя', '<NEWLINE>', '<NEWL

In [39]:
import json

responses_parsed = eval(responses[0])

In [40]:
responses_parsed

['Был вечерок, и скользкие товы',
 '<NEWLINE>',
 'Вращались и кружились в вабе:',
 '<NEWLINE>',
 'Все мимзи были бороговы,',
 '<NEWLINE>',
 'И момы раты выгребали',
 '<NEWLINE>',
 '<NEWLINE>',
 '“Остерегайся Жаббервока, мой сын',
 '<NEWLINE>',
 'Челюсти, что кусают, когти, что ловят',
 '<NEWLINE>',
 'Остерегайся птицы Юб-юб, и избегай',
 '<NEWLINE>',
 'Зловещего Бандершнатча',
 '”',
 '<NEWLINE>',
 '<NEWLINE>',
 'Он взял свой ворпаловый меч в руки;',
 '<NEWLINE>',
 'Долго искал он врага манксома—',
 '<NEWLINE>',
 'Так отдыхал он у дерева Тамтам',
 '<NEWLINE>',
 'И стоял некоторое время в раздумье',
 '<NEWLINE>',
 '<NEWLINE>',
 'И, стоя в уффишной задумчивости,',
 '<NEWLINE>',
 'Жаббервок, с глазами пламени,',
 '<NEWLINE>',
 'Пришуршал сквозь тулги лес,',
 '<NEWLINE>',
 'И бурчал, как приходил',
 '<NEWLINE>',
 '<NEWLINE>',
 'Раз, два',
 'Раз, два',
 'И взад и вперед',
 '<NEWLINE>',
 'Ворпаловый клинок шмыг-шмяк',
 '<NEWLINE>',
 'Он оставил его мертвым, и с его головой',
 '<NEWLINE>',
 'О